# Run experiments to "Validate Mathis model at Edge, Core and Intermediate Scale"

## Set up your FABRIC environment


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

In [ ]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [ ]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test'

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [ ]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Setup the scripts and modules

First, we will upload all the scripts we need to analyze the results for this experiment. This needs to be done only once before start of experiment.

In [ ]:
for n in sender_nodes:
    n.upload_file('process_cwn_file.py','process_cwn_file.py')
    n.upload_file('process_iperf_normal.py','process_iperf_normal.py')
    n.upload_file('mathis_sender.py','mathis_sender.py')
    n.upload_file("cwn.sh", "cwn.sh")

Also we need sklearn to process the downloaded files, so install the library in the environment.This needs to be done only once before start of experiment.

In [ ]:
pip install scikit-learn

## Generate flows

### Set experiment parameters

>cca, delay, test_duration, num_servers, flows, interval, omit

cca is the congestion control algorithm (reno for this experiment)

delay is the delay to be set at the receiver (20 ms,100 ms,200 ms)

test_duration is the time for which to send the iperf3 flows (10800 is used for these experiments).

num_servers is the number of ports to be opened on each receiver. 
CoreScale : 100,3000 and 500 ports
EdgeScale : 1,3 and 5 ports
IntermediateScale: 10, 30 and 50 ports

flows is the number of parallel flows to be send from each port. It is set to 1 for all the experiments.

interval is the periodic time interval to save the result by iperf3.

omit is the starting n seconds to ignore the iperf values.(set to 0 for all the experiments. Max value=300)

In [ ]:
cca="reno"
delay=200
test_duration=3600
num_servers=100
flows=1
interval=0.01
omit=0

### Get queue statistics on the router before experiment

In [ ]:
router_node.execute("tc -p -s -d -j qdisc show dev "+router_egress_name +" >tc_before.txt")

## Remove existing result files from the hosts

In [ ]:
for n in receiver_nodes:
    n.execute("rm -f 60*")

for n in sender_nodes:
    n.execute("rm -f sender*")
    n.execute("rm -f data*")
    n.execute("rm -f packet*")
    n.execute("rm -f output*")

Check if the files are removed from the senders and receivers

### Set delay on the receiver

> Values: 20ms, 100ms, 200ms

Now set up delay on the receiver interface:

First delete any existing queue (don't worry if there is an error, it means there was not!)

In [ ]:
for n in receiver_nodes:
    receiver_inf=n.get_interface(network_name= "link-receiver")
    receiver_inf_name = receiver_inf.get_device_name()
    n.execute("sudo tc qdisc del dev " + receiver_inf_name + " root netem")
    n.execute("sudo tc qdisc add dev " + receiver_inf_name + " root netem delay " + str(delay) +"ms limit 1000000")
    

### Start parallel servers on the receivers

Now start the `iperf3` flows:

In [ ]:
for i, n in enumerate(receiver_nodes):
    n.execute("sudo killall iperf3")
    n.execute_thread(f'chmod +x iperf-parallel-servers.sh && bash iperf-parallel-servers.sh '+str(num_servers))

In [ ]:
#check if the required number of ports are opened
for n in receiver_nodes:
    n.execute("ls -1 | wc -l")

### Start parallel clients on the senders

In [ ]:
import time
                                  
for i, n in enumerate(sender_nodes):
    n.execute("sudo killall iperf3")
    n.execute_thread(f'chmod +x iperf-parallel-senders.sh && bash iperf-parallel-senders.sh 10.10.2.1'+str(i)+" "+str(num_servers)+" "+\
                     str(test_duration)+" "+cca+" "+str(flows)+" "+str(interval)+" "+str(omit))
    n.execute_thread(f'chmod +x cwn.sh && bash cwn.sh 10.10.2.1'+str(i))
                                       
time.sleep(test_duration+300)             

### Get queue statistics on the router after experiment

In [ ]:
router_node.execute("tc -p -s -d -j qdisc show dev "+router_egress_name +" >tc_after.txt")

## Analyze the results

Calculate sum of bandwidth, square of sum of bandwidth, count of flows and jfi:

Calculate packet drop rate

In [ ]:
#To get packet dropped:
(drop_before,err_drop_before)=router_node.execute("tail --lines=10 tc_before.txt| grep '\"drops\":' | awk '{print $2}' |cut -d ',' -f1")

#To get packets sent
(sent_before,err_sent_before)=router_node.execute("tail --lines=10 tc_before.txt| grep '\"packets\":' | awk '{print $2}' |cut -d ',' -f1")

#To get packet dropped:
(drop_after,err_drop_after)=router_node.execute("tail --lines=10 tc_after.txt| grep '\"drops\":' | awk '{print $2}' |cut -d ',' -f1")

#To get packets sent
(sent_after, err_sent_after)=router_node.execute("tail --lines=10 tc_after.txt| grep '\"packets\":' | awk '{print $2}' |cut -d ',' -f1")

#Calculate packet drop rate:

n_seg_dropped=int(drop_after)-int(drop_before)

n_seg_sent=int(sent_after)-int(sent_before)
drop_rate=float(n_seg_dropped)/float(n_seg_sent)

print("packet drop before running experiment: "+ str(drop_before))
print("packet sent before running experiment: " + str(sent_before))
print("packet drop after running experiment: "+ str(drop_after))
print("packet sent after  running experiment: " + str(sent_after))
print("packet sent: " + str(n_seg_sent))
print("packet dropped: " + str(n_seg_dropped))
print("packet drop rate: " + str(drop_rate))



Run the data processing scripts on each sender to get packet loss, congestion window halving events and rtt from iperf3 and ss output.

In [ ]:
for i,n in enumerate(sender_nodes):
    n.execute_thread(f'chmod +x process_cwn_file.py && python3 process_cwn_file.py '+str(i) )
    n.execute_thread(f'chmod +x process_iperf_normal.py && python3 process_iperf_normal.py '+str(i)+\
                                     " "+str(num_servers)+" "+str(test_duration)+" "+cca+" "+str(flows))



Run mathis_sender.py script on each sender to get packet loss rate and cwnd halving rate of each flow and save the output to packet_loss_iperf{i}.csv file

In [ ]:
import time
while True:
    i=False
    time.sleep(100)
    for n in sender_nodes:
        (res,err)=n.execute("pgrep -af python")
        if "python3 process_" in res:
            i=True
    if i:
        continue
    else:
        break
for i,n in enumerate(sender_nodes):
    n.execute('chmod +x mathis_sender.py && python3 mathis_sender.py '+str(i))

Download all the packet_loss_iperf(i).csv file to the environment.

In [ ]:
for i,n in enumerate(sender_nodes):
    n.download_file("/home/fabric/work/IMC6/fabric-notebooks/packet_loss"+str(i)+".csv", "/home/ubuntu/packet_loss"+str(i)+".csv")

Process the downloaded file to get the Mathis constant. Final output is saved to output_mathis_C.csv file.

In [ ]:
import pandas as pd
import numpy as np
import csv
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages
import sys
import os


dat_exp = pd.concat([pd.read_csv("packet_loss"+str(i)+".csv") for i in range(n_endpoints) ], ignore_index=True)

dat_exp = dat_exp.assign(p_router_drop = n_seg_dropped/n_seg_sent )

coef_retrans_ss    = LinearRegression(fit_intercept = False).fit(
  ( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_ss_retrans'].values) ) ).values.reshape(-1,1), 
  dat_exp['bitrate']*1000.0
).coef_
coef_retrans_iperf = LinearRegression(fit_intercept = False).fit(
  ( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_iperf_retrans'].values) ) ).values.reshape(-1,1), 
  dat_exp['bitrate']*1000.0
).coef_
coef_cwnd_halve    = LinearRegression(fit_intercept = False).fit(
  ( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_cwnd_halve'].values) ) ).values.reshape(-1,1), 
  dat_exp['bitrate']*1000.0
).coef_
coef_router_dropped = LinearRegression(fit_intercept = False).fit(
  ( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_router_drop'].values) ) ).values.reshape(-1,1), 
  dat_exp['bitrate']*1000.0
).coef_
print(coef_retrans_ss, coef_retrans_iperf, coef_cwnd_halve, coef_router_dropped)

print( dat_exp.agg({'port': ['count'], 'bitrate': ['sum'], 'data_seg': ['sum'], 'retrans_ss': ['sum'], 'retrans_iperf': ['sum'], 'cwnd_halve': ['sum'], 'rtt': ['mean'] }) )


p=dat_exp['port'].aggregate('count')
bw=dat_exp['bitrate'].aggregate('sum')
seg=dat_exp['data_seg'].aggregate('sum')
retrans_ss_sum=dat_exp['retrans_ss'].aggregate('sum')
retrans_iperf_sum=dat_exp['retrans_iperf'].aggregate('sum')
cwn_halve_sum=dat_exp['cwnd_halve'].aggregate('sum')
rtt_mean=dat_exp['rtt'].aggregate('mean')


output_filename='output_mathis_C.csv'
if not os.path.isfile(output_filename):
    with open(output_filename, 'a', newline='') as csvfile:
      writer = csv.writer(csvfile)
      header = 'time_duration', 'ports', 'Base_RTT(ms)', 'BW', 'total_data_seg_out','total_cwnd_half', 'total_retransmission_ss',\
        'total_retransmission_iperf', 'total_retransmission_ss/total_cwnd_half', 'total_retransmission_iperf/total_cwnd_half',\
        'C_ss', 'C_iperf', 'C_cwnd', 'C_router', 'router_dropped', 'router_sent', 'router_dropped/total_cwnd_half', \
        'mdape_ss', 'mdape_iperf', 'mdape_cwnd', 'mdape_router'
      writer.writerow(header)
      
        
x_retrans_ss=( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_ss_retrans'].values) ) ).values.reshape(-1,1)
x_retrans_iperf=( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_iperf_retrans'].values) ) ).values.reshape(-1,1)
x_cwnd=( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_cwnd_halve'].values) ) ).values.reshape(-1,1)
x_router=( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_router_drop'].values) ) ).values.reshape(-1,1)
predicted_bw_ss    = LinearRegression(fit_intercept = False).fit(
  x_retrans_ss, 
  dat_exp['bitrate']*1000.0
).predict(x_retrans_ss)
predicted_bw_iperf = LinearRegression(fit_intercept = False).fit(
  x_retrans_iperf, 
  dat_exp['bitrate']*1000.0
).predict(x_retrans_iperf)
predicted_bw_cwnd    = LinearRegression(fit_intercept = False).fit(
  x_cwnd, 
  dat_exp['bitrate']*1000.0
).predict(x_cwnd)
predicted_bw_router = LinearRegression(fit_intercept = False).fit(
  x_router, 
  dat_exp['bitrate']*1000.0
).predict(x_router)

mdape_ss=np.median((np.abs(np.subtract(dat_exp['bitrate']*1000.0, predicted_bw_ss)/ (dat_exp['bitrate']*1000.0)))) * 100
mdape_iperf=np.median((np.abs(np.subtract(dat_exp['bitrate']*1000.0, predicted_bw_iperf)/ (dat_exp['bitrate']*1000.0)))) * 100
mdape_cwnd=np.median((np.abs(np.subtract(dat_exp['bitrate']*1000.0, predicted_bw_cwnd)/ (dat_exp['bitrate']*1000.0)))) * 100
mdape_router=np.median((np.abs(np.subtract(dat_exp['bitrate']*1000.0, predicted_bw_router)/ (dat_exp['bitrate']*1000.0)))) * 100

with open(output_filename, 'a', newline='') as csvfile:
  writer = csv.writer(csvfile)   
  columns = test_duration, p, delay, bw, seg, cwn_halve_sum, retrans_ss_sum,retrans_iperf_sum, retrans_ss_sum/cwn_halve_sum,\
    retrans_iperf_sum/cwn_halve_sum, coef_retrans_ss[0], coef_retrans_iperf[0], coef_cwnd_halve[0], coef_router_dropped[0],\
    n_seg_dropped, n_seg_sent, n_seg_dropped/cwn_halve_sum, mdape_ss, mdape_iperf,  mdape_cwnd, mdape_router
  writer.writerow(columns)


Save the linear regression plots to a pdf.

In [ ]:
with PdfPages("linear_reg_plot.pdf") as pdf:
  plt.rcParams['figure.figsize'] = (8,6)

  plt.scatter(x=x_retrans_ss, y=dat_exp['bitrate']*1000.0, color='C4', alpha=1, s=10, label='actual values')
  plt.scatter(x=x_retrans_ss, y=predicted_bw_ss, color='C3',  alpha=1, s=10, label='predicted_values')
  plt.plot(x_retrans_ss, predicted_bw_ss, color='C2', linewidth=0.5, label='fit')
  plt.xlabel("x=mss/rtt*sqrt(packet_loss_rate)") 
  plt.ylabel("y=bandwidth(bits/sec)")
  plt.title("Method-1 calculation of packet_loss rate using data_seg_out from ss and retrans from ss data")
  plt.legend()
  pdf.savefig()  # saves the current figure into a pdf page
  plt.show()
  plt.close()

  plt.scatter(x=x_retrans_iperf, y=dat_exp['bitrate']*1000.0, color='C4', s=10, label='actual values')
  plt.scatter(x=x_retrans_iperf, y=predicted_bw_iperf, color='C3', s=10, label='predicted_values')
  plt.plot(x_retrans_iperf, predicted_bw_iperf, color='C2', linewidth=0.5, label='fit')
  plt.xlabel("x=mss/rtt*sqrt(packet_loss_rate)") 
  plt.ylabel("y=bandwidth(bits/sec)")
  plt.title("Method-2: calculation of packet_loss rate using data_seg_out from ss and retrans from iperf3 data")
  plt.legend()
  pdf.savefig()  # saves the current figure into a pdf page
  plt.show()
  plt.close()


  plt.scatter(x=x_cwnd, y=dat_exp['bitrate']*1000.0, color='C4', alpha=1, s=10, label='actual values')
  plt.scatter(x=x_cwnd, y=predicted_bw_cwnd, color='C3',  alpha=1, s=10, label='predicted_values')
  plt.plot(x_cwnd, predicted_bw_cwnd, color='C2', linewidth=0.5, label='fit')
  plt.xlabel("x=mss/rtt*sqrt(packet_loss_rate)")
  plt.ylabel("y=bandwidth(bits/sec)")
  plt.title("Method-3: calculation of packet_loss rate using data_seg_out from ss and cwnd from iperf3 data")
  plt.legend()
  pdf.savefig()  # saves the current figure into a pdf page
  plt.show()
  plt.close()

  plt.scatter(x=x_router, y=dat_exp['bitrate']*1000.0, color='C4', alpha=1, s=10, label='actual values')
  plt.scatter(x=x_router, y=predicted_bw_router, color='C3',  alpha=1, s=10, label='predicted_values')
  plt.plot(x_router, predicted_bw_router, color='C2', linewidth=0.5, label='fit')
  plt.xlabel("x=mss/rtt*sqrt(packet_loss_rate)")
  plt.ylabel("y=bandwidth(bits/sec)")
  plt.title("Method-4: calculation of packet_loss rate using packet drop rate at the router")
  plt.legend()
  pdf.savefig()  # saves the current figure into a pdf page
  plt.show()
  plt.close()
 

Get JFI and save it to file 'jfi.csv'

In [ ]:
import pandas as pd
import numpy as np
import csv
import sys
import os


sq_y_values=(dat_exp['bitrate']*dat_exp['bitrate']).aggregate('sum')
   

jfi_filename='jfi.csv'
if not os.path.isfile(jfi_filename):
    with open(jfi_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        header ='CCA', 'Duration of Expt(sec)', 'Base RTT(ms)', 'Total Bandwidth(Kbps)', 'Sum of sq of BW', 'Flow Count', 'JFI'
        writer.writerow(header)
    
with open(jfi_filename, 'a', newline='') as csvfile:
  writer = csv.writer(csvfile)
  columns = cca, test_duration, delay, bw , sq_y_values, p, (bw*bw)/(sq_y_values*p)
  writer.writerow(columns)

Remove the transfered files after data analysis and/or downloading to your local system.

In [ ]:
%rm pac*

In [ ]:
%rm linear*